In [ ]:
# GPTCache :一个致力于为存储 LLM 响应构建语义缓存的开源项目。

In [1]:
# !pip install gptcache --quiet


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

cf_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('CF_API_BASE'),
    openai_api_key=os.getenv('CF_API_TOKEN'),
    model_name="@cf/meta/llama-3-8b-instruct",
    temperature=0,
    streaming=True
)

In [10]:
qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)


In [13]:
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

精确匹配

In [2]:
import langchain
from gptcache import Cache
from gptcache.manager.factory import manager_factory
from gptcache.processor.pre import get_prompt
from gptcache.adapter.api import init_similar_cache
from langchain.cache import GPTCache
import hashlib


def get_hashed_name(name):
    return hashlib.sha256(name.encode()).hexdigest()


def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    cache_obj.init(
        pre_embedding_func=get_prompt,
        data_manager=manager_factory(manager="map", data_dir=f"map_cache_{hashed_llm}"),
    )


langchain.llm_cache = GPTCache(init_gptcache)


In [5]:
question = "What is cache eviction policy?"

In [6]:
%%time

cf_llm_openai.invoke(question)

CPU times: user 697 ms, sys: 45.2 ms, total: 743 ms
Wall time: 32.1 s


AIMessage(content='Cache eviction policy, also known as cache replacement policy, is a set of rules that defines how a cache (a small, fast memory location) handles the situation where it needs to remove an item to make room for a new one. This is known as cache eviction. The goal of the policy is to ensure that the cache remains as effective as possible, by minimizing the number of times items need to be retrieved from slower, larger storage devices (such as hard drives).\n\nThere are several common cache eviction policies:\n\n1. **FIFO (First-In-First-Out)**: The policy removes the item that has been in the cache the longest (FIFO) to make room for the new item. This is the simplest policy to implement.\n\n2. **LRU (Least Recently Used)**: The policy removes the item that has not been accessed for the longest time (LRU) to make room for the new item. This policy is more effective than FIFO as it favors the items that are most likely to be reused.\n\n3. **LFU (Least Frequently Used)**

In [7]:
%%time

cf_llm_openai.invoke(question)

CPU times: user 1.57 ms, sys: 2.24 ms, total: 3.81 ms
Wall time: 3.6 ms


AIMessage(content='Cache eviction policy, also known as cache replacement policy, is a set of rules that defines how a cache (a small, fast memory location) handles the situation where it needs to remove an item to make room for a new one. This is known as cache eviction. The goal of the policy is to ensure that the cache remains as effective as possible, by minimizing the number of times items need to be retrieved from slower, larger storage devices (such as hard drives).\n\nThere are several common cache eviction policies:\n\n1. **FIFO (First-In-First-Out)**: The policy removes the item that has been in the cache the longest (FIFO) to make room for the new item. This is the simplest policy to implement.\n\n2. **LRU (Least Recently Used)**: The policy removes the item that has not been accessed for the longest time (LRU) to make room for the new item. This policy is more effective than FIFO as it favors the items that are most likely to be reused.\n\n3. **LFU (Least Frequently Used)**

In [8]:
%%time

cf_llm_openai.invoke('What is cache eviction   policy?')

CPU times: user 425 ms, sys: 35.9 ms, total: 460 ms
Wall time: 33.6 s


AIMessage(content='Cache eviction policy, also known as cache replacement policy, is a strategy used by a cache (a small, fast memory that stores frequently-used data) to decide which items to remove (evict) when the cache is full and new data needs to be stored. The goal of cache eviction policy is to minimize the number of cache misses (when data is not found in the cache) and maintain optimal performance.\n\nThere are several cache eviction policies, each with its own trade-offs and characteristics. Here are some common ones:\n\n1. **LRU (Least Recently Used)**: Removes the item that has not been accessed for the longest time. This policy assumes that frequently used items will be accessed again soon.\n2. **FIFO (First-In-First-Out)**: Removes the item that was added to the cache first. This policy is simple but can be less efficient than LRU.\n3. **LRU-K (Least Recently Used with a cost associated with each item)**: Assigns a cost to each item based on its size or access frequency.

Similar Match
相似匹配

In [14]:
from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from langchain.cache import GPTCache
import hashlib


def get_hashed_name(name):
    return hashlib.sha256(name.encode()).hexdigest()


# cf embedding 当前不支持
def init_gptcache(cache_obj: Cache, llm: str):
    hashed_llm = get_hashed_name(llm)
    init_similar_cache(cache_obj=cache_obj, data_dir=f"similar_cache_{hashed_llm}", embedding=embedding)


langchain.llm_cache = GPTCache(init_gptcache)

In [15]:
%%time

qw_llm_openai.invoke(question)

AttributeError: 'CloudflareWorkersAIEmbeddings' object has no attribute 'dimension'

In [ ]:
%%time

qw_llm_openai.invoke("What is cache eviction   policy?")

In [ ]:
%%time

qw_llm_openai.invoke("Give me a peach")

In [ ]:
%%time

qw_llm_openai.invoke("Give me 2 peaches")